In [ ]:
import time

import matplotlib.pyplot as plt
import numpy as np
import sklearn.datasets
import tensorflow as tf

# Prints numpy arrays nicer
np.set_printoptions(precision=2, suppress=True, linewidth=100)

In [ ]:
boston_housing = sklearn.datasets.load_boston()
# Uncomment the following line for a description of the dataset.
# print(boston_housing['DESCR'])
xs = boston_housing.data
ys = boston_housing.target

standardize_xs = True
if standardize_xs:
    xs = (xs - np.mean(xs, axis=0)) / np.std(xs, axis=0)
ys = np.expand_dims(ys, -1)  # Adding the extra axis enables tf.matmul later

data = list(zip(xs, ys))

# Perform 60% / 40% training/test split
split_index = int(len(data) * 0.6)
train_data = data[:split_index]
test_data = data[split_index:]
print('Num training examples:', len(train_data))
print('Num testing examples:', len(test_data))

In [ ]:
# Hyperparameters
learning_rate = 0.005 if not standardize_xs else 0.05
num_epochs = 500
num_features = len(train_data[0][0])
num_hidden = 20
batch_size = 10

# Model Definition
batch_x = tf.placeholder(tf.float32, shape=[None, num_features])
batch_y = tf.placeholder(tf.float32, shape=[None, 1])

dense1 = tf.layers.Dense(num_hidden, activation=tf.nn.sigmoid)
dense2 = tf.layers.Dense(1)

h = dense1(batch_x)
y_prediction = dense2(h)

loss = tf.losses.mean_squared_error(labels=batch_y, predictions=y_prediction)
train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    # Training
    time_before = time.time()
    losses = []  # Storing losses so we can plot them later
    for epoch in range(num_epochs):
        np.random.shuffle(train_data)
        cumulative_loss = 0
        for i in range(0, len(train_data), batch_size):
            _batch_x, _batch_y = zip(*train_data[i:i + batch_size])
            _loss, _train_op = sess.run(
                (loss, train_op),
                feed_dict={batch_x: _batch_x, batch_y: _batch_y})
            cumulative_loss += _loss * len(_batch_x)
        average_loss = cumulative_loss / len(train_data)
        if epoch % 25 == 24:
            print('Epoch: {}, Loss: {}'.format(epoch + 1, average_loss))
        losses.append(average_loss)
    time_after = time.time()
    print('Training took {:.2f}s.'.format(time_after - time_before))

    # Prediction
    train_xs, train_ys = zip(*train_data)
    train_ys_prediction = sess.run(y_prediction, feed_dict={batch_x: train_xs})

    test_xs, test_ys = zip(*test_data)
    test_ys_prediction = sess.run(y_prediction, feed_dict={batch_x: test_xs})

In [ ]:
plt.figure(dpi=150)
plt.title('Loss over Time')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.plot(range(len(losses)), losses, color='#458588')
plt.show()

In [ ]:
train_mean_squared_error = np.mean((train_ys - train_ys_prediction) ** 2)
test_mean_squared_error = np.mean((test_ys - test_ys_prediction) ** 2)

print('Mean Squared Error on Training data:', train_mean_squared_error)
print('Mean Squared Error on Testing data:', test_mean_squared_error)

plt.figure(dpi=150)
plt.title('Actual vs Predicted Data Points: Training Set')
plt.xlabel('Actual House Price')
plt.ylabel('Predicted House Price')
plt.plot([min(train_ys), max(train_ys)], [min(train_ys), max(train_ys)],
         color='#1D2021', linestyle='--')
plt.scatter(train_ys, train_ys_prediction, color='#458588')
plt.show()

plt.figure(dpi=150)
plt.title('Actual vs Predicted Data Points: Testing Set')
plt.xlabel('Actual House Price')
plt.ylabel('Predicted House Price')
plt.plot([min(test_ys), max(test_ys)], [min(test_ys), max(test_ys)],
         color='#1D2021', linestyle='--')
plt.scatter(test_ys, test_ys_prediction, color='#CC241D')
plt.show()